In [1]:
import pandas as pd

df = pd.read_csv('./asr_bengali/utt_spk_text.tsv', sep='\t')
df.columns = ["id", "hash", "text"]
df.head()

,id,hash,text
0,000039928e,976b1,এ ধরণের কার্ড নিয়ে
1,00005debc7,f83df,হতে উপার্জিত অর্থ
2,00009e687c,9813c,হাসির বিষয় হয়েই আছে
3,00012843bc,7ec1c,সার্ক দেশগুলোতে
4,00012880b1,e43d4,সহজেই যাওয়া যায়


In [3]:
txt = df.loc[df['id'] == "00012843bc", ["text"]]
txt.values[0][0]

'সার্ক দেশগুলোতে'